In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import math

# Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_spectrogram_X_and_Y(X_vector_len=X_vector_len, split_noise=True, take_log=True, split=0.85)

SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


# Train
* Hmm, sparse matrix doesnt work

In [3]:
# Need this while testing graph so it gets autoloaded
from kaggle_speech_recog.graphs.useful_tf_graph import *
from kaggle_speech_recog.graphs.spectrogram_multi_lstm import *

In [4]:
graph_name = 'SpectrogramMultiLSTM'  # Specify graph class name

g_cnfg = Config('graph_02_dropout075_on_input')  # Specify graph configuration name
g_cnfg.X_img_h = XY_train_valid[0].shape[1]
g_cnfg.X_img_w = XY_train_valid[0].shape[2]
g_cnfg.Y_vector_len = XY_train_valid[1].shape[1]

#g_cnfg.conv1_n_filters = 16
#g_cnfg.conv1_kernel_size = 5
#g_cnfg.conv1_mp_size = 5
#g_cnfg.conv1_mp_strides = 2

#g_cnfg.conv2_n_filters = 32
#g_cnfg.conv2_kernel_size = 3
#g_cnfg.conv2_mp_size = 3
#g_cnfg.conv2_mp_strides = 2

g_cnfg.lstm_state_sizes = [100, 200]
#g_cnfg.lstm_dropout = {'in': 'y', 'out': 'n', 'state': 'n'}

#g_cnfg.X_conv_h = math.ceil(math.ceil(g_cnfg.X_img_h / g_cnfg.conv1_mp_strides) / g_cnfg.conv2_mp_strides)
#g_cnfg.X_conv_w = math.ceil(math.ceil(g_cnfg.X_img_w / g_cnfg.conv1_mp_strides) / g_cnfg.conv2_mp_strides)
#g_cnfg.n_conv_flat = g_cnfg.X_conv_h * g_cnfg.X_conv_w * g_cnfg.conv2_n_filters
#g_cnfg.n_flat = g_cnfg.n_conv_flat + g_cnfg.lstm_state_sizes[-1]

g_cnfg.n_hidden1 = 50
#g_cnfg.n_hidden2 = 50

g_cnfg.lr_initial = 0.1
g_cnfg.lr_decay_steps = 1000000
g_cnfg.lr_decay_rate = 0.7  # 1.0 means no decay

t_cnfg = Config('run_02_split_noise')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.dropout_keep_prob = 0.75
t_cnfg.log_every = 50
t_cnfg.print_every = math.ceil(XY_train_valid[0].shape[0]/t_cnfg.batch_size)  # every epoch
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [ ]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, XY_train_valid)

name: graph_02_dropout075_on_input
X_img_h: 129
X_img_w: 71
Y_vector_len: 31
lr_decay_rate: 0.7
lr_decay_steps: 1,000,000
lr_initial: 0.1
lstm_state_sizes: [100, 200]
n_hidden1: 50

name: run_02_split_noise
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.75
early_stopping_patience: 0.1
le: LabelEncoder()
log_dir: logs
log_every: 50
max_step: 100,000,000
n_ave_ll_valid: 8
print_every: 370
start_step_early_stopping: 100,000,000
tb_dir: tensorboard

SpectrogramMultiLSTM_graph_02_dropout075_on_input_run_02_split_noise
Training starts @ 12/12/2017 23:01:55
Epoch 1 Step 1 Best model saved @ 12/12/2017 23:02:00 [Ave valid] 3.434
Epoch 1 Step 1 ends @ 12/12/2017 23:02:00 [Train] 3.434, 3.3% [Valid] 4.0% [Ave valid] 3.434
Epoch 1 Step 50 Best model saved @ 12/12/2017 23:02:09 [Ave valid] 3.427
Epoch 1 Step 100 Best model saved @ 12/12/2017 23:02:19 [Ave valid] 3.416
Epoch 1 Step 150 Best model saved @ 12/12/2017 23:02:28 [Ave valid] 3.399
Epoch 1 Step 200 Best model saved @ 12/12/2017

Epoch 16 Step 5,920 Best model saved @ 12/12/2017 23:21:22 [Ave valid] 0.399
Epoch 16 Step 5,920 ends @ 12/12/2017 23:21:22 [Train] 0.866, 75.0% [Valid] 86.5% [Ave valid] 0.399
Epoch 17 Step 6,000 Best model saved @ 12/12/2017 23:21:39 [Ave valid] 0.397
Epoch 17 Step 6,050 Best model saved @ 12/12/2017 23:21:49 [Ave valid] 0.396
Epoch 17 Step 6,100 Best model saved @ 12/12/2017 23:21:58 [Ave valid] 0.394
Epoch 17 Step 6,150 Best model saved @ 12/12/2017 23:22:08 [Ave valid] 0.392
Epoch 17 Step 6,200 Best model saved @ 12/12/2017 23:22:17 [Ave valid] 0.389
Epoch 17 Step 6,250 Best model saved @ 12/12/2017 23:22:27 [Ave valid] 0.385
Epoch 17 Step 6,290 ends @ 12/12/2017 23:22:35 [Train] 0.063, 100.0% [Valid] 81.8% [Ave valid] 0.408
Epoch 18 Step 6,650 Best model saved @ 12/12/2017 23:23:45 [Ave valid] 0.375
Epoch 18 Step 6,660 Best model saved @ 12/12/2017 23:23:49 [Ave valid] 0.371
Epoch 18 Step 6,660 ends @ 12/12/2017 23:23:49 [Train] 0.936, 75.0% [Valid] 88.4% [Ave valid] 0.371
Epoch 